In [1]:
from jsonformer.pydantic_parser import create_scheme
from jsonformer.format import highlight_values
from jsonformer.main import Jsonformer
from typing import Literal
from pydantic import BaseModel

In [2]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
import torch


checkpoint = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)


tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
audio_brands = Literal[
    "JBL",
    "Bose",
    "Sony",
    "Harman Kardon"
]

class Performance(BaseModel):
    engine: str
    horsepower: float
    topSpeed: float

class Safety(BaseModel):
    airbags: int 
    parkingSensors: bool 
    laneAssist: bool

class Audio(BaseModel):
    brand: audio_brands
    speakers: int
    hasBluetooth: bool

class Features(BaseModel):
    audio: Audio
    safety: Safety
    performance: Performance

class Car(BaseModel):
    make: str
    model: str
    year: int 
    colors: list[str]
    features: Features

class Owner(BaseModel):
    firstName: str
    lastName: str
    age: int

class CarObject(BaseModel):
    car: Car
    owner: Owner

car_object = create_scheme(CarObject)
car_object

{'type': 'object',
 'properties': {'car': {'type': 'object',
   'properties': {'make': {'type': 'string'},
    'model': {'type': 'string'},
    'year': {'type': 'number'},
    'colors': {'type': 'array', 'items': {'type': 'string'}},
    'features': {'type': 'object',
     'properties': {'audio': {'type': 'object',
       'properties': {'brand': {'type': 'enum',
         'values': ['JBL', 'Bose', 'Sony', 'Harman Kardon']},
        'speakers': {'type': 'number'},
        'hasBluetooth': {'type': 'boolean'}}},
      'safety': {'type': 'object',
       'properties': {'airbags': {'type': 'number'},
        'parkingSensors': {'type': 'boolean'},
        'laneAssist': {'type': 'boolean'}}},
      'performance': {'type': 'object',
       'properties': {'engine': {'type': 'string'},
        'horsepower': {'type': 'number'},
        'topSpeed': {'type': 'number'}}}}}}},
  'owner': {'type': 'object',
   'properties': {'firstName': {'type': 'string'},
    'lastName': {'type': 'string'},
    'age'

In [4]:
builder = Jsonformer(
    model=model,
    tokenizer=tokenizer,
    json_schema=car_object,
    prompt="generate an example car",
    max_string_token_length=20,
)

print("Generating...")
output = builder()

highlight_values(output)

Generating...
{
  car: {
    make: "Toyota",
    model: "Camry",
    year: 2018.0,
    colors: [
      "Silver",
      "Black",
      "Red",
      "Blue",
      "Green",
      "Yellow",
      "Purple",
      "Orange",
      "Gray",
      "White"
    ],
    features: {
      audio: {
        brand: "JBL",
        speakers: 10.0,
        hasBluetooth: True
      },
      safety: {
        airbags: 6.0,
        parkingSensors: False,
        laneAssist: False
      },
      performance: {
        engine: "4.0L V6",
        horsepower: 301.0,
        topSpeed: 130.0
      }
    }
  },
  owner: {
    firstName: "John",
    lastName: "Doe",
    age: 32.0
  }
}
